# Choose mutations to retain and sites to saturate
Choose mutations and sites to target with saturating mutagenesis.

Import Python modules:

In [ ]:
import os
import re

import altair as alt

import numpy

import pandas as pd

import yaml

_ = alt.data_transformers.enable("default", max_rows=None)

Read configuration:

In [ ]:
if not os.path.isfile("config.yaml"):
    os.chdir("../")  # if running interactively in a subdirectory
    
with open("config.yaml") as f:
    config = yaml.safe_load(f)

Get the count types and their thresholds:

In [ ]:
thresholds = config["mutation_retain_thresholds"]

count_types = sorted(thresholds)

site_thresholds = config["site_saturation_threshold"]

site_count_types = sorted(site_thresholds)

assert set(site_count_types).issubset(count_types)

Read mutation statistics and make sure they have all count types expected:

In [ ]:
mutation_stats = pd.read_csv(config["mutation_stats"]).query("mutant_aa != wildtype_aa")

if not set(count_types).issubset(mutation_stats.columns):
    raise ValueError(f"some {count_types=} not in {mutation_stats.columns=}")
    
mutation_stats_tidy = mutation_stats.melt(
    id_vars=[
        "reference_site",
        "sequential_site",
        "reference_aa",
        "wildtype_aa",
        "mutant_aa",
    ],
    value_vars=count_types,
    var_name="count_type",
    value_name="count",
)

Specify sites to allow deletions:

In [ ]:
sites_to_allow_deletions = {
    r
    for site_range in config["sites_to_allow_deletions"]
    for r in range(site_range[0], site_range[1] + 1)
}

print(f"{len(sites_to_allow_deletions)} have allowed deletions")

mutation_stats_tidy = mutation_stats_tidy.assign(
    allow_deletion=lambda x: x["reference_site"].isin(sites_to_allow_deletions)
)

Specify mutations to include and sites to saturate:

In [ ]:
mutations_to_include = set(config["mutations_to_include"])

print(f"{len(mutations_to_include)} mutations to include")

sites_to_saturate = set(config["sites_to_saturate"])

print(f"{len(sites_to_saturate)} sites for saturating mutagenesis")

saturate_diffs_from_reference = config["saturate_diffs_from_reference"]

print(f"Also using {saturate_diffs_from_reference=}")

mutation_stats_tidy = mutation_stats_tidy.assign(
    _mut=lambda x: x["reference_site"].astype(str) + x["mutant_aa"],
    mutation_to_include=lambda x: x["_mut"].isin(mutations_to_include),
    site_to_saturate=lambda x: x["reference_site"].isin(sites_to_saturate),
).drop(columns="_mut")

Add site statistics:

In [ ]:
mutation_stats_tidy = (
    mutation_stats_tidy
    .merge(
        mutation_stats_tidy
        .groupby(["count_type", "reference_site"], as_index=False)
        .aggregate(site_count=pd.NamedAgg("count", "sum")),
        on=["reference_site", "count_type"],
        validate="many_to_one",
    )
)

Plot ranks of mutation stats.
For plotting purposes, mutations with zero counts are assigned a value of 0.1:

In [ ]:
ranks = (
    mutation_stats_tidy.assign(
        rank=lambda x: x.groupby("count_type")["count"].transform(
            "rank", method="first", ascending=False,
        ),
        count=lambda x: x["count"].clip(lower=0.1),
    )
)

In [ ]:
# this needs to be true for code below to work
if not all(re.fullmatch("\w+", count_type) for count_type in count_types):
    raise ValueError(f"{count_types=} not all alphanumeric without spaces")

# create parameters to adjust thresholds
params = [
    alt.param(
        name=count_type,
        value=threshold,
        bind=alt.binding_range(
            name=f"{count_type} mutation threshold",
            min=1,
            max=threshold * 10,
            step=1,
        ),
    )
    for count_type, threshold in thresholds.items()
]

site_params = [
    alt.param(
        name=f"site_{count_type}",
        value=threshold,
        bind=alt.binding_range(
            name=f"{count_type} site threshold",
            min=1,
            max=threshold * 10,
            step=10 if threshold >=100 else 1,
        ),
    )
    for count_type, threshold in site_thresholds.items()
]
    
# altair transform calculate which mutations to retain based on count
retain_by_count_str = " | ".join(
    f"((datum.count >= {count_type}) & (datum.count_type == '{count_type}'))"
    for count_type in thresholds
)

saturate_by_count_str = " | ".join(
    f"((datum.site_count >= site_{count_type}) & (datum.count_type == '{count_type}'))"
    for count_type in site_thresholds
)

retained_selection = alt.selection_point(fields=["retained"], bind="legend")

mutant_aas = sorted(mutation_stats["mutant_aa"].unique())
mutant_aa_selection = alt.selection_point(
    fields=["mutant_aa"],
    bind=alt.binding_select(
        options=[None, *mutant_aas],
        labels=["all", *mutant_aas],
        name="mutant amino acid",
    ),
)

saturated_selection = alt.selection_point(
    fields=["saturated"],
    bind=alt.binding_select(
        options=[None, "saturated", "not saturated"],
        labels=["all", "saturated", "not saturated"],
        name="saturated sites",
    ),
)

mutation_selection = alt.selection_point(
    on="mouseover", fields=["reference_site", "mutant_aa"], empty=False,
)

saturate_diffs_from_reference_selection = alt.param(
    value=saturate_diffs_from_reference,
    bind=alt.binding_radio(
        options=[True, False],
        name="saturate sites that differ from reference",
    ),
)

base_chart = (
    alt.Chart(ranks)
    .transform_calculate(
        retain_by_count=retain_by_count_str,
        saturate_by_count=saturate_by_count_str,
        diff_from_ref=alt.datum.reference_aa != alt.datum.wildtype_aa,
        saturate=(
            alt.datum.saturate_by_count
            | alt.datum.site_to_saturate
            | (alt.datum.diff_from_ref & saturate_diffs_from_reference_selection)
        ),
        retain=(
            (
                alt.datum.retain_by_count
                & ((alt.datum.mutant_aa != "-") | alt.datum.allow_deletion)
            )
            | alt.datum.mutation_to_include
            | (
                alt.datum.saturate
                & (alt.datum.mutant_aa != "-")
                & (alt.datum.mutant_aa != "*")
            )
        ),
        retained=alt.expr.if_(alt.datum.retain, "retained", "not retained"),
        saturated=alt.expr.if_(alt.datum.saturate, "saturated", "not saturated"),
    )
    .add_params(
        *params,
        *site_params,
        retained_selection,
        mutation_selection,
        mutant_aa_selection,
        saturated_selection,
        saturate_diffs_from_reference_selection,
    )
)

rank_chart = (
    base_chart
    .transform_filter(retained_selection)
    .transform_filter(mutant_aa_selection)
    .transform_filter(saturated_selection)
    .encode(
        x=alt.X(
            "rank",
            scale=alt.Scale(
                nice=False,
                domainMin=ranks["rank"].min(),
                domainMax=ranks["rank"].max(),
            ),
            title="mutation rank",
        ),
        y=alt.Y("count", scale=alt.Scale(type="log"), title=None),
        color=alt.Color(
            "retained:N",
            scale=alt.Scale(domain=["retained", "not retained"]),
            title="click / shift-click to select",
        ),
        size=alt.condition(mutation_selection, alt.value(60), alt.value(7)),
        strokeWidth=alt.condition(mutation_selection, alt.value(2), alt.value(0)),
        row=alt.Row("count_type", title=None),
        tooltip=[
            "reference_site",
            "sequential_site",
            "reference_aa",
            "wildtype_aa",
            "mutant_aa",
            "count",
            "rank",
            "retained:N",
            "saturated:N",
        ],
    )
    .mark_point(filled=True, stroke="black")
    .resolve_scale(y="independent")
    .properties(
        width=500,
        height=150,
        title=alt.TitleParams("ranked mutation counts"),
    )
)

bar_chart = (
    base_chart
    .transform_aggregate(
        retain="sum(retain)",
        groupby=["reference_site", "mutant_aa"],
    )
    .transform_calculate(
        retained=alt.expr.if_(alt.datum.retain, "retained", "not retained"),
    )
    .encode(
        x=alt.X("retained:N", title=None),
        y=alt.Y("count(reference_site):Q", title="number of mutations"),
        tooltip=[
            alt.Tooltip("count(reference_site):Q", title="number of mutations"),
            "retained:N",
        ],
    )
    .mark_bar(color="black")
    .properties(
        width=alt.Step(15),
        height=200,
        title=alt.TitleParams("mutations retained"),
    )
)

site_base_chart = (
    base_chart
    .transform_aggregate(
        saturate="sum(saturate)",
        groupby=["reference_site", "sequential_site", "wildtype_aa", "reference_aa"],
    )
    .transform_calculate(
        saturated=alt.expr.if_(alt.datum.saturate, "saturated", "not saturated"),
    )
)

site_bar_chart = (
    site_base_chart
    .encode(
        x=alt.X("saturated:N", title=None),
        y=alt.Y("count(reference_site):Q", title="number of sites"),
        tooltip=[
            alt.Tooltip("count(reference_site):Q", title="number of sites"),
            "saturated:N",
        ],
    )
    .mark_bar(color="black")
    .properties(
        width=alt.Step(15),
        height=200,
        title=alt.TitleParams("sites saturated"),
    )
)

site_saturated_chart = (
    site_base_chart
    .encode(
        x=alt.X(
            "reference_site",
            title="site (reference numbering)",
            scale=alt.Scale(zero=False, nice=False),
            axis=alt.Axis(grid=False),
        ),
        y=alt.Y(
            "saturated:N",
            title=None,
            scale=alt.Scale(domain=["saturated", "not saturated"]),
        ),
        tooltip=[
            "reference_site",
            "sequential_site",
            "wildtype_aa",
            "reference_aa",
            "saturated:N",
        ],
    )
    .mark_point(size=15, filled=True, color="black")
    .properties(
        width=700,
        height=150,
        title="saturated sites",
    )
)

chart = ((bar_chart & site_bar_chart) | rank_chart) & site_saturated_chart

print(f"Saving chart to {config['mutations_to_make_chart']}")
chart.save(config["mutations_to_make_chart"])

chart

Now actually get lists of the mutations to make and sites to saturate given the default thresholds:

In [ ]:
to_retain = (
    mutation_stats
    .query("wildtype_aa != mutant_aa")
    .assign(
        retain_by_count=lambda x: numpy.logical_or.reduce(
            [x[count_type] >= threshold for count_type, threshold in thresholds.items()]
        ),
        saturate_by_count=lambda x: numpy.logical_or.reduce(
            [
                x.groupby("reference_site")[count_type].transform("sum") >= threshold
                for count_type, threshold in site_thresholds.items()
            ]
        ),
        site_to_saturate=lambda x: x["reference_site"].isin(sites_to_saturate),
        saturate=lambda x: (
            x["saturate_by_count"]
            | x["site_to_saturate"]
            | ((x["reference_aa"] != x["wildtype_aa"]) & saturate_diffs_from_reference)
        ),
        retain=lambda x: (
            (
                x["retain_by_count"]
                & (
                    (x["mutant_aa"] != "-")
                    | x["reference_site"].isin(sites_to_allow_deletions)
                )
            ) | (x["reference_site"].astype(str) + x["mutant_aa"]).isin(
                    mutations_to_include
                )
            | (x["saturate"] & ~x["mutant_aa"].isin(["-", "*"]))
        ),
    )
)

targeted_mutations = (
    to_retain
    .query("retain")
    [["reference_site", "sequential_site", "wildtype_aa", "reference_aa", "mutant_aa"]]
)

print(
    f"Writing {len(targeted_mutations)} mutations to retain "
    f"to {config['targeted_mutations']}"
)
targeted_mutations.to_csv(config["targeted_mutations"], index=False)

saturated_sites = (
    to_retain
    .query("saturate")
    [["reference_site", "sequential_site", "wildtype_aa", "reference_aa"]]
    .drop_duplicates()
)

print(
    f"Writing {len(saturated_sites)} sites to saturate "
    f"to {config['saturated_sites']}"
)
saturated_sites.to_csv(config["saturated_sites"], index=False)